# 🥋 FalkorDB Agent with Pydantic AI - UFC Knowledge Graph Demo

This notebook demonstrates how to create an intelligent **UFC-focused AI agent** using **FalkorDB** and **Pydantic AI** that can query a comprehensive UFC knowledge graph powered by the **GraphRAG SDK**.

## What You'll Learn

- How to set up a Pydantic AI agent with custom tools
- How to integrate FalkorDB with the GraphRAG SDK
- How to create a knowledge graph search tool for your agent
- How to build an interactive chat interface with streaming responses
- How to handle dependencies and error management in agent workflows

## Prerequisites

Before running this notebook, ensure you have:

- A running FalkorDB instance with UFC data loaded (see [demo-ufc.ipynb](./demo-ufc.ipynb))
- OpenAI API key for the language model
- Python environment with required packages (installed below)

## 🔧 Installation Requirements

This notebook requires three main packages. Install them with the cell below:

### Required Packages:
- **GraphRAG SDK**: Provides FalkorDB integration, Rich console formatting, OpenAI client, and graph operations
- **Pydantic AI**: Modern AI agent framework with type safety and structured data handling  
- **Gradio**: Web interface framework for creating interactive chat applications


**Note**: The GraphRAG SDK includes most dependencies we need, so the installation is streamlined!

In [1]:
!pip install graphrag_sdk --quiet
!pip install pydantic-ai --quiet
!pip install gradio --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
deepeval 2.6.7 requires anthropic<0.50.0,>=0.49.0, but you have anthropic 0.55.0 which is incompatible.
google-genai 1.22.0 requires httpx<1.0.0,>=0.28.1, but you have httpx 0.27.2 which is incompatible.
mistralai 1.8.2 requires httpx>=0.28.1, but you have httpx 0.27.2 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
deepeval 2.6.7 requires anthropic<0.50.0,>=0.49.0, but you have anthropic 0.55.0 which is incompatible.
ollama 0.2.1 requires httpx<0.28.0,>=0.27.0, but you have httpx 0.28.1 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following depend

## 1. Import Required Libraries

First, let's import all the necessary libraries for our FalkorDB and Pydantic AI agent.

In [2]:
import gradio as gr
from typing import List
from falkordb import FalkorDB
from dataclasses import dataclass
from __future__ import annotations
from pydantic import BaseModel, Field
from graphrag_sdk import KnowledgeGraph
from pydantic_ai import Agent, RunContext
from graphrag_sdk.ontology import Ontology
from graphrag_sdk.models.litellm import LiteModel
from graphrag_sdk.chat_session import ChatSession
from pydantic_ai.models.openai import OpenAIModel
from pydantic_ai.providers.openai import OpenAIProvider
from graphrag_sdk.model_config import KnowledgeGraphModelConfig

## 2. Load Environment Variables and Configuration

Load environment variables for API keys and database connection parameters.

In [ ]:
# Configuration parameters
MODEL_CHOICE = 'gpt-4o-mini'
OPENAI_API_KEY = "your-openai-api-key-here"  # Replace with your actual API key or use os.getenv

# FalkorDB connection parameters
FALKORDB_HOST = "localhost" # Replace with your FalkorDB host
FALKORDB_PORT = 6379  # Default port for FalkorDB
FALKORDB_USERNAME = "your-falkordb-username"  # can be None if not required
FALKORDB_PASSWORD = "your-falkordb-password"  # can be None if not required

GRAPH_NAME = "ufc"

## 3. Set Up the LLM Model

Configure the OpenAI model that will power our Pydantic AI agent.

In [4]:
def get_model():
    """Configure and return the LLM model to use."""
    return OpenAIModel(MODEL_CHOICE, provider=OpenAIProvider(api_key=OPENAI_API_KEY))

## 4. Connect to FalkorDB and Load Ontology

Establish connection to FalkorDB and load the existing UFC knowledge graph ontology.

In [5]:
# Connect to FalkorDB
print("🔌 Connecting to FalkorDB...")
db = FalkorDB(host=FALKORDB_HOST, port=FALKORDB_PORT, username=FALKORDB_USERNAME, password=FALKORDB_PASSWORD)
graph = db.select_graph(GRAPH_NAME)

# Load ontology from existing graph
try:
    ontology = Ontology.from_kg_graph(graph)
    print("✅ Loaded ontology from existing knowledge graph.")
    print(f"   Entities: {len(ontology.entities) if ontology.entities else 0}")
    print(f"   Relations: {len(ontology.relations) if ontology.relations else 0}")
except Exception as e:
    print(f"⚠️  Could not load ontology from existing graph: {str(e)}")

🔌 Connecting to FalkorDB...
✅ Loaded ontology from existing knowledge graph.
   Entities: 5
   Relations: 7


## 5. Initialize KnowledgeGraph and ChatSession

Set up the GraphRAG SDK components for knowledge graph interaction.

In [6]:
# Initialize LiteModel for GraphRAG SDK (by default - gpt-4.1)
model_falkor = LiteModel()

# Initialize model configuration
model_config = KnowledgeGraphModelConfig.with_model(model_falkor)

# Initialize KnowledgeGraph
print("🔗 Initializing KnowledgeGraph client...")
kg_client = KnowledgeGraph(
    name=GRAPH_NAME,
    model_config=model_config,
    ontology=ontology,
    host=FALKORDB_HOST,
    port=FALKORDB_PORT,
    username=FALKORDB_USERNAME,
    password=FALKORDB_PASSWORD
)

# Create a chat session for agent queries (replaces cypher_session)
chat_session = kg_client.chat_session()
print("✅ ChatSession created successfully!")

🔗 Initializing KnowledgeGraph client...
✅ ChatSession created successfully!


## 6. Define Agent Dependencies

Create a dependencies class to pass the ChatSession to our agent.

In [7]:
@dataclass
class FalkorDependencies:
    """Dependencies for the Falkor agent."""
    chat_session: ChatSession

print("📦 Agent dependencies defined!")

📦 Agent dependencies defined!


## 7. Create the Pydantic AI Agent

Define our main agent with a comprehensive system prompt for UFC knowledge graph querying.

In [8]:
# Create the Falkor agent
falkor_agent = Agent(
    get_model(),
    system_prompt="""You are a knowledge graph assistant that helps users query a FalkorDB knowledge graph.

When a user provides ANY input (questions, entity names, keywords, or statements), you MUST use the search_falkor tool with the EXACT, COMPLETE user input as the query parameter. Do not modify, shorten, or extract keywords from the user's input.

The knowledge graph system is designed to handle:
- Full questions: "Who is Salsa Boy?"
- Entity names: "Salsa Boy"
- Keywords: "fighters", "matches", "UFC"
- Statements: "Show me information about recent fights"
- Any other text input

The tool will return:
- A Cypher query that was generated to search the graph (automatically adapted to the input type)
- Context data extracted from the graph using that query

After receiving the results, explain what the Cypher query does and interpret the context data to provide a helpful answer. Focus on the entities, relationships, and graph patterns found in the results. If the input was just an entity name, provide comprehensive information about that entity and its connections.
Try to be concise but thorough in your explanations, ensuring the user understands the graph data and its implications.""",
    deps_type=FalkorDependencies
)

print("🤖 Falkor Agent created successfully!")

🤖 Falkor Agent created successfully!


## 8. Define the Search Result Model

Create a Pydantic model to structure the results from our FalkorDB search tool.

This model ensures type safety and clear data structure for the agent's search results.

In [9]:
class FalkorSearchResult(BaseModel):
    """Model representing a search result from FalkorDB."""
    cypher: str = Field(description="The generated Cypher query")
    context: str = Field(description="The extracted context from the knowledge graph")

print("📊 Search result model defined!")

📊 Search result model defined!


## 9. Register the Search Tool

Create and register the main tool that allows our agent to search the FalkorDB knowledge graph.

In [10]:
@falkor_agent.tool
async def search_falkor(ctx: RunContext[FalkorDependencies], query: str) -> List[FalkorSearchResult]:
    """Search the FalkorDB knowledge graph with the given query - returns only cypher and context.
    
    Args:
        ctx: The run context containing dependencies
        query: The search query to find information in the knowledge graph
        
    Returns:
        A list of search results containing cypher queries and context that match the query
    """
    # Access the ChatSession from dependencies
    chat_session = ctx.deps.chat_session
    
    try:
        # Use the chat session's generate_cypher_query method to get cypher and context
        # This method returns (context, cypher) tuple
        context, cypher = chat_session.generate_cypher_query(query)
        
        # Format the result to match the expected interface
        formatted_result = FalkorSearchResult(
            cypher=cypher or '',
            context=context or ''
        )
        
        return [formatted_result]
    except Exception as e:
        # Log the error
        print(f"Error searching FalkorDB: {str(e)}")
        raise

print("🔍 Search tool registered with agent!")

🔍 Search tool registered with agent!


## 10. Gradio Chat Function

Create the Gradio integration function for web-based chat interface.

In [11]:
async def gradio_chat_function(message, history):
    """
    Gradio chat function that processes user messages and returns agent responses.
    
    Args:
        message (str): The user's message
        history (List): Chat history from Gradio
        
    Returns:
        str: The agent's response
    """
    deps = FalkorDependencies(chat_session=chat_session)
    
    try:
        # Convert Gradio history to our message format
        message_history = []
        for user_msg, assistant_msg in history:
            if user_msg:
                message_history.append({"role": "user", "content": user_msg})
            if assistant_msg:
                message_history.append({"role": "assistant", "content": assistant_msg})
        
        # Get response from agent
        result = await falkor_agent.run(
            message,
            deps=deps
        )
        
        return result.data
        
    except Exception as e:
        return f"❌ Error: {str(e)}"

print("🌐 Gradio chat function defined!")

🌐 Gradio chat function defined!


## 11. Create Gradio Web Interface

Build an elegant web-based chat interface using Gradio for easy interaction with the UFC knowledge graph agent.

In [12]:
def create_gradio_interface():
    """Create a clean, elegant Gradio chat interface."""
    
    def chat(message, history):
        """Handle chat interactions."""
        if not message.strip():
            return history, ""
        
        try:
            import asyncio
            response = asyncio.run(gradio_chat_function(message, history))
            history.append((message, response))
            return history, ""
        except Exception as e:
            history.append((message, f"❌ Error: {str(e)}"))
            return history, ""
    
    # Create interface with minimal, clean design
    with gr.Blocks(title="🥋 UFC Knowledge Graph Agent", theme=gr.themes.Soft()) as interface:
        
        gr.Markdown("# 🥋 UFC Knowledge Graph Agent\n### Ask anything about UFC fighters, fights, and events!")
        
        chatbot = gr.Chatbot(height=400, show_copy_button=True)
        
        with gr.Row():
            msg = gr.Textbox(placeholder="Ask me about UFC...", scale=4, container=False)
            submit = gr.Button("🚀 Ask", variant="primary")
            clear = gr.Button("🗑️", variant="secondary")
        
        # Example questions
        gr.Examples([
            "Who is Salsa Boy?",
            "Show me recent UFC fights",
            "Which fighters have the most wins?",
            "What are the UFC weight classes?"
        ], msg)
        
        # Event handling
        submit.click(chat, [msg, chatbot], [chatbot, msg])
        msg.submit(chat, [msg, chatbot], [chatbot, msg])
        clear.click(lambda: [], outputs=chatbot)
    
    return interface

# Create and launch interface
gradio_interface = create_gradio_interface()
print("🌐 Gradio interface created!")

🌐 Gradio interface created!


/tmp/ipykernel_210415/1172376482.py:23: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(height=400, show_copy_button=True)


## 12. Launch the Interface

Launch the Gradio web interface and start chatting with the UFC knowledge graph agent!

In [13]:
# Launch the interface
print("🚀 Launching UFC Knowledge Graph Agent...")
gradio_interface.launch(share=True, inbrowser=True)

🚀 Launching UFC Knowledge Graph Agent...
* Running on local URL:  http://127.0.0.1:7860
* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://79db0b9f73212d5729.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
* Running on public URL: https://79db0b9f73212d5729.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/tmp/ipykernel_210415/2419845010.py:29: DeprecationWarning: `result.data` is deprecated, use `result.output` instead.
  return result.data
